------------------------------------


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
bajka = '/content/drive/MyDrive/uni/uam/uczenie_glebokie/alternatywna-bajka-o-sierotce-marysi.txt'
finetune = '/content/drive/MyDrive/uni/uam/uczenie_glebokie/finetuning.csv'

In [ ]:
bajka

'/content/drive/MyDrive/uni/uam/uczenie_glebokie/alternatywna-bajka-o-sierotce-marysi.txt'

--------------------

In [ ]:
!pip install faiss-cpu sentence-transformers transformers accelerate chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.8/70.8 kB 8.8 MB/s eta 0:00

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 22.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import torch
import faiss
import numpy as np
import pandas as pd
from datasets import Dataset
from sentence_transformers import SentenceTransformer
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    pipeline,
    EarlyStoppingCallback,
    AutoModelForSeq2SeqLM
)

In [ ]:
# model_name = "allegro/herbert-base-cased"
model_name = "allegro/plt5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# Dodaj specjalne tokeny jeśli ich nie ma
special_tokens = {
    "pad_token": "[PAD]",
    "eos_token": "</s>",
    "bos_token": "<s>",
    "unk_token": "[UNK]",
}
tokenizer.add_special_tokens(special_tokens)
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/658 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

Embedding(50003, 768)

In [ ]:
#########################
# Chunkowanie
#########################
def chunk_text(text, chunk_size=500, overlap=50):

    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start += (chunk_size - overlap)
    return chunks

with open(bajka, "r", encoding="utf-8") as f:
    raw_text = f.read()

chunks = chunk_text(raw_text, chunk_size=500, overlap=50)
print(f"Liczba chunków: {len(chunks)}")

Liczba chunków: 50


In [ ]:
#########################
# Embedding + FAISS
#########################
embedding_model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embedder = SentenceTransformer(embedding_model_name)

# Zamiana chunków na wektory
embeddings = embedder.encode(chunks, convert_to_numpy=True)

dimension = embeddings.shape[1]  # np. 384
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)  # Dodajemy wektory do indexu

def retrieve_chunks(query, top_k=3):
    query_vec = embedder.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, top_k)
    relevant = []
    for idx in indices[0]:
        relevant.append(chunks[idx])
    return relevant

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Zakładamy, że finetune_data.csv ma kolumny: 'prompt' i 'output'
df_finetune = pd.read_csv(finetune)
dataset_finetune = Dataset.from_pandas(df_finetune)

# Podział na train / val
dataset_finetune = dataset_finetune.train_test_split(test_size=0.1, seed=42)
train_data = dataset_finetune["train"]
val_data   = dataset_finetune["test"]

In [ ]:
print(train_data.column_names)

['prompt', 'output']


In [ ]:
!pip install sacremoses

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 12.2 MB/s eta 0:00:00


In [ ]:
def tokenize_function(examples):
    # Wyodrębniamy kontekst z promptu (zakładając, że prompt zawiera "KONTEKST:" na początku)
    contexts = []
    questions = []

    for prompt in examples["prompt"]:
        # Rozdziel prompt na kontekst i pytanie
        parts = prompt.split("Pytanie:")
        if len(parts) == 2:
            context = parts[0].replace("KONTEKST:", "").strip()
            question = parts[1].strip()
        else:
            # Jeśli nie ma wyraźnego podziału, traktuj cały prompt jako pytanie
            context = ""
            question = prompt

        contexts.append(context)
        questions.append(question)

    # Przygotuj prompt w odpowiednim formacie
    prompts = [
        f"Kontekst: {context}\nPytanie: {question}\nOdpowiedź:"
        for context, question in zip(contexts, questions)
    ]

    # Tokenizacja inputów
    model_inputs = tokenizer(
        prompts,
        truncation=True,
        max_length=512,
        padding="max_length",
        return_tensors="pt"
    )

    # Tokenizacja odpowiedzi (labels)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["output"],
            truncation=True,
            max_length=128,
            padding="max_length",
            return_tensors="pt"
        )

    model_inputs["labels"] = labels["input_ids"]
    # Zamień padding na -100 aby model je ignorował
    model_inputs["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in label]
        for label in model_inputs["labels"]
    ]

    return model_inputs

# Aplikuj nową funkcję tokenizacji
train_data = train_data.map(
    tokenize_function,
    batched=True,
    remove_columns=train_data.column_names
)
val_data = val_data.map(
    tokenize_function,
    batched=True,
    remove_columns=val_data.column_names
)

train_data.set_format("torch")
val_data.set_format("torch")

Map:   0%|          | 0/84 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
# Dodaj własną klasę trenera
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

training_args = TrainingArguments(
    output_dir="plt5_finetuned",
    num_train_epochs=50,  # więcej epok dla małego datasetu
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,  # zwiększona akumulacja gradientów
    warmup_ratio=0.25,  # dłuższy warmup
    learning_rate=2e-5,  # mniejszy learning rate dla stabilniejszego treningu
    weight_decay=0.02,  # zwiększona regularyzacja
    evaluation_strategy="steps",
    eval_steps=5,  # częstsza ewaluacja
    save_strategy="steps",
    save_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=False,  # wyłącz dla stabilności
    logging_steps=5
)


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None): # Add num_items_in_batch argument
        outputs = model(**inputs)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

# Inicjalizacja trenera z early stopping
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

trainer.add_callback(early_stopping)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer

In [ ]:
trainer.train()
trainer.save_model("SierotkaMarysia")
tokenizer.save_pretrained("SierotkaMarysia")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
5,243.563600,20.809429
10,205.660400,20.583637
15,200.778400,20.112881
20,199.173000,18.884163
25,180.061600,17.536959
30,184.620800,16.018143
35,189.743500,14.779752
40,168.238000,14.311178
45,161.387900,13.090347
50,144.542700,11.255438


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


('SierotkaMarysia/tokenizer_config.json',
 'SierotkaMarysia/special_tokens_map.json',
 'SierotkaMarysia/spiece.model',
 'SierotkaMarysia/added_tokens.json',
 'SierotkaMarysia/tokenizer.json')

In [ ]:
finetuned_model = AutoModelForSeq2SeqLM.from_pretrained("SierotkaMarysia")
finetuned_tokenizer = AutoTokenizer.from_pretrained("SierotkaMarysia")
finetuned_model.to("cuda" if torch.cuda.is_available() else "cpu")

def generate_finetuned_answer(question, top_k=3, max_len=512):
    context_chunks = retrieve_chunks(question, top_k=top_k)
    context_text = "\n".join(context_chunks)

    prompt = f"Kontekst: {context_text}\nPytanie: {question}\nOdpowiedź:"

    inputs = finetuned_tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=max_len,
        padding=True
    ).to(finetuned_model.device)

    outputs = finetuned_model.generate(
        **inputs,
        max_length=128,  # krótsze odpowiedzi
        num_beams=3,     # beam search
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.2,  # kara za powtórzenia
        early_stopping=True
    )

    return finetuned_tokenizer.decode(outputs[0], skip_special_tokens=True)

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers


In [ ]:
def chat_rag_finetuned():
    print("Zadaj pytanie, wpisz 'exit' aby zakończyć.\n")
    while True:
        user_input = input("PYTANIE: ")
        if user_input.strip().lower() in ["exit", "quit", "koniec"]:
            print("Koniec.")
            break

        answer = generate_finetuned_answer(user_input, top_k=3)
        print(f"\nODPOWIEDŹ:\n{answer}\n")

# Uruchom chat
chat_rag_finetuned()

Zadaj pytanie, wpisz 'exit' aby zakończyć.

PYTANIE: KONTEKST:\nProfesor miał specjalny komputer.\nPytanie:Jak wyglądał komputer kwantowy profesora?

ODPOWIEDŹ:
規 komputer kwantowy profesora miał specjalny komputer.

PYTANIE: Marysia

ODPOWIEDŹ:
規, który nie był w stanie ogarnąć całego świata.

PYTANIE: profesor

ODPOWIEDŹ:
規 złości?

PYTANIE: ile?

ODPOWIEDŹ:
規, że w tym świecie nic innego nie zaznała.

PYTANIE: Buty po dziadku?

ODPOWIEDŹ:
規 po dziadku?

PYTANIE: a co?

ODPOWIEDŹ:
規, co się dzieje w świecie, w którym nie ma komputerów kwantowych.

PYTANIE: co się dzieje w świecie, w którym nie ma komputerów kwantowych?

ODPOWIEDŹ:
規, że nie ma komputerów kwantowych, które mogłyby cię przenieść.

PYTANIE: koniec
Koniec.
